# Step 2 ES Feature Individual Level
## Reviws and Rating Extraction 

Right now we recover a historic snapshot of ratings.   
In the future we can add NLP pieces(such as count of certain words). 

Input:
- close rate, monthly:  installer_close_rates.csv 
- reviews, by piece: installer_review_data_20180410.csv 
- reviews sentiment score, per each review 

Output: 
- 'oct29_closerate_withratingcounts.csv' 

Uplate: June25 , 2020




## Get the Input data 

In [1]:
pwd

'C:\\Users\\Herbie Huang\\Google Drive\\Current_SolarResearch\\working_june2020\\solar_project\\1_code'

In [2]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import os
 

## Input : installer_close_rates and installer_review_data_20180410

In [3]:

close=pd.read_csv('../0_data/Lock_ES_RawData/installer_close_rates.csv',sep=',')

#%%
reviews=pd.read_csv('../0_data/Lock_ES_RawData/installer_review_data_20180410.csv',sep=',',escapechar='\\')


Close Rate (ES Installers)

In [5]:
#%%
#close rate have many NAs, and year_month are not integer 

close = close[np.isfinite(close['Year_month'])]
close['Year_month']=pd.to_numeric(arg=close['Year_month'],downcast='integer')
#%%
close['year_month']=close['Year_month']
#%% relace na with 0 
close[['quotes_count','winning_quotes_count']]=close[['quotes_count','winning_quotes_count']].fillna(0)
#%%
close['year_month']=pd.to_datetime(close['Year_month'],format='%Y%m',errors='coerce')
close['closeraterecord_id']=close.index
#now the year _month colun are set to the correct time format 

In [6]:
close[50:60]

,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
50,50,424,201710,238.0,4.0,2017-10-01,50
51,51,424,201711,211.0,3.0,2017-11-01,51
52,52,424,201712,134.0,4.0,2017-12-01,52
53,53,424,201801,227.0,0.0,2018-01-01,53
54,54,424,201802,245.0,0.0,2018-02-01,54
55,55,424,201803,306.0,3.0,2018-03-01,55
56,56,424,201804,78.0,0.0,2018-04-01,56
57,57,551,201312,1.0,1.0,2013-12-01,57
58,58,551,201403,2.0,0.0,2014-03-01,58
59,59,551,201404,1.0,0.0,2014-04-01,59


Note this still has gaps in time. When we append the reviews score it will also have gap. 

## Input: sent_score from data pipeline 

In [12]:
reviews_sent_scores=pd.read_csv('../2_pipeline/reviews_sent_score_jan16_2020.csv')

In [14]:
reviews_sent_scores=reviews_sent_scores[['id','installer_id','overall_rating','sent_score']]

## Merge Reviews data with sentiment scores 

In [15]:
reviews=pd.merge(left=reviews,right=reviews_sent_scores,on=['id','installer_id','overall_rating'],how='inner')

In [16]:
reviews.tail()

,id,installer_id,subject,body,active,authenticated,date_created,overall_rating,number_quotes_submitted,number_winning_quotes,sent_score
3382,3392,3247,AVOID this company,"Terrible service, difficult to deal with, wait...",1,1,2018-04-09 15:34:12,1,0,0,-0.6808
3383,3393,21242,Altair is amazing! Look no further.,This was our second solar installation. After ...,1,1,2018-04-09 16:05:47,5,842,14,0.9704
3384,3394,21269,Installation of Solar Pqnels,In April 2017 Solar Five Company installed 32 ...,1,1,2018-04-09 19:49:58,5,276,3,0.6597
3385,3395,21711,"Professional, cost-effective and easy for me",Pinnacle installed our solar panels and I coul...,1,0,2018-04-09 21:57:33,5,10,0,0.6588
3386,3396,21866,Quality Install,I signed up with America Green back in Februar...,1,1,2018-04-10 19:04:47,4,0,0,0.8689


## Reshape and Create the installer-monthly reviews rating and sentiment record 

In [19]:
close['closeraterecord_id'][:6]

0    0
1    1
2    2
3    3
4    4
5    5
Name: closeraterecord_id, dtype: int64

In [18]:
reviews.tail().sent_score.mean()

0.49540000000000006

### Helper Function: count occurances of X star reviews in a list 

In [20]:
def countX(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count

### Loop through the closerate data frame to collect the corresponding rating and sent_score 
For every piece of closerate record ( shaped like installer-month) 
- temp_seedate collects the year_month of that piece of record 
- temp_installer records the installer_id 
- temp_reviews_step1, temp_reviews_step2 : find the reviews set that belonged to that instaler, before and including that month 
- temp_count5, temp_count4..temp_count1: count how many pieces of 5-star , 4-star..1-star reviews in this set 
- temp_sent_scores, temp_sent_avg: find the set of relevant reviews and their sent_score - temp_sent_score. Take the average - temp_sent_avg 
- temp_dataexp: collect this row of data : i - closerrate index. temp_count5/4/3/2/1 , temp_avg_sent_score 

In [21]:
#%%
#%%
reviews['date_created']=pd.to_datetime(reviews['date_created'])
#%%
allratings=[]
collect=close
avgrating=[]
count=[]
data_exp=[]
#%%

for i in close['closeraterecord_id']:
    temp_seedate=close['year_month'].loc[i]
    temp_installer=close['installer_id'].loc[i]
    temp_reviews_step1=reviews[reviews['installer_id']==temp_installer]
    # step 1 is to slice the reviews that only relevant to this installer as temp_installer is the id 
    temp_reviews_step2=temp_reviews_step1[reviews['date_created']<=temp_seedate]
    #step 2 is to slice the reviews that happened before the "see date" (temp_seedate)
    temp_ratings=temp_reviews_step2['overall_rating']
    # print(temp_ratings)
    temp_sent_scores=temp_reviews_step2.sent_score
    temp_count5=countX(temp_ratings,5)
    temp_count4=countX(temp_ratings,4) 
    temp_count3=countX(temp_ratings,3)
    temp_count2=countX(temp_ratings,2)
    temp_count1=countX(temp_ratings,1)
    temp_avg_sent_score=temp_sent_scores.mean()
   # temp_countstars=[temp_count5,temp_count4,temp_count3,temp_count2,temp_count1]
   # count.append(temp_countstars)
    temp_dataexp=[i,temp_count5,temp_count4,temp_count3,temp_count2,temp_count1,temp_avg_sent_score]
    data_exp.append(temp_dataexp)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [22]:
data_exp[-5:-1]

[[6406, 6, 0, 0, 0, 0, 0.56655],
 [6407, 0, 0, 0, 0, 0, nan],
 [6408, 2, 0, 0, 0, 0, 0.655],
 [6409, 0, 0, 0, 0, 0, nan]]

In [23]:
data_exp_df=pd.DataFrame(data=data_exp,columns=['i', 'five','four','three','two','one','avg_sent_score'])
#%%

Verify that data_exp_df is the same structure and length as closerate data 

In [24]:
data_exp_df.tail()

,i,five,four,three,two,one,avg_sent_score
6406,6406,6,0,0,0,0,0.56655
6407,6407,0,0,0,0,0,NaN
6408,6408,2,0,0,0,0,0.65500
6409,6409,0,0,0,0,0,NaN
6410,6410,3,0,0,0,0,0.69600


In [25]:
close.tail()

,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
6406,6406,22992,201804,10.0,0.0,2018-04-01,6406
6407,6407,22997,201803,28.0,0.0,2018-03-01,6407
6408,6408,22997,201804,4.0,0.0,2018-04-01,6408
6409,6409,23027,201803,2.0,0.0,2018-03-01,6409
6410,6410,23027,201804,15.0,0.0,2018-04-01,6410


In [26]:
merged_new_closerecord=pd.merge(left=data_exp_df,right=close,left_on='i',right_on='closeraterecord_id',how='inner')

In [27]:
merged_new_closerecord.head()

,i,five,four,three,two,one,avg_sent_score,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
0,0,0,0,0,0,0,NaN,0,108,201506,27.0,0.0,2015-06-01,0
1,1,0,0,0,0,0,NaN,1,108,201601,12.0,0.0,2016-01-01,1
2,2,0,0,0,0,0,NaN,2,108,201602,3.0,0.0,2016-02-01,2
3,3,0,0,0,0,0,NaN,3,363,201603,20.0,0.0,2016-03-01,3
4,4,0,0,0,0,0,NaN,4,363,201604,8.0,0.0,2016-04-01,4


In [28]:
pwd

'C:\\Users\\Herbie Huang\\Google Drive\\Current_SolarResearch\\working_june2020\\solar_project\\1_code'

## Output: closerate_withratingcounts_sent_score to 2_pipeline

In [30]:
merged_new_closerecord.to_csv('../2_pipeline/closerate_withratingcounts_sent_score.csv')